## Optional: Advanced SQL (Bonus)

This notebook contains additional SQL analysis beyond the core dashboard scope.
It is not required to understand the Power BI report, but demonstrates deeper SQL skills
(window functions, segmentation, ranking, and trend analysis).


In [21]:
import pandas as pd 
import sqlite3 
conn = sqlite3.connect("../data/processed/HGFN.db")

## 1) Revenue & Profitability (SKU + Category)
Goal: Identify what drives revenue and profit, and whether results are concentrated in a small set of SKUs.


In [22]:
q1 = """
SELECT
  o.sku_id,
  s.category,
  s.sub_category,
  SUM(o.quantity * s.price) AS total_revenue,
  SUM(o.quantity * (s.price - s.cost)) AS gross_profit,
  SUM(o.quantity) AS total_units
FROM orders_clean o
JOIN sku_clean s ON s.sku_id = o.sku_id
GROUP BY o.sku_id, s.category, s.sub_category
ORDER BY total_revenue DESC
LIMIT 20;
"""
pd.read_sql(q1, conn)



,sku_id,category,sub_category,total_revenue,gross_profit,total_units
0,SKU00828,Pet Supplies,Other Pets,36667.82,21561.06,86
1,SKU03177,Health & Beauty,Personal Care,36586.40,21553.60,76
2,SKU07906,Pet Supplies,Cat,36098.16,19825.68,84
3,SKU07467,Health & Beauty,Supplements,35360.10,17783.10,90
4,SKU02833,Health & Beauty,Skincare,34788.39,16691.22,91
5,SKU07500,Pet Supplies,Cat,34737.78,20489.64,73
6,SKU06075,Home & Kitchen,Cookware,34629.30,19628.10,90
7,SKU00976,Sports,Outdoor,33703.81,18715.67,83
8,SKU04021,Grocery,Beverages,33584.48,18530.24,79
9,SKU09711,Apparel,Women,33301.50,19839.75,75


In [23]:
q2 = """
WITH sku_rev AS (
  SELECT
    o.sku_id,
    SUM(o.quantity * s.price) AS sku_revenue
  FROM orders_clean o
  JOIN sku_clean s ON s.sku_id = o.sku_id
  GROUP BY o.sku_id
),
tot AS (
  SELECT SUM(sku_revenue) AS total_revenue FROM sku_rev
)
SELECT
  sku_id,
  sku_revenue,
  ROUND(1.0 * sku_revenue / (SELECT total_revenue FROM tot), 4) AS revenue_share
FROM sku_rev
ORDER BY sku_revenue DESC
LIMIT 20;
"""
pd.read_sql(q2, conn)


,sku_id,sku_revenue,revenue_share
0,SKU00828,36667.82,0.0007
1,SKU03177,36586.40,0.0007
2,SKU07906,36098.16,0.0007
3,SKU07467,35360.10,0.0006
4,SKU02833,34788.39,0.0006
5,SKU07500,34737.78,0.0006
6,SKU06075,34629.30,0.0006
7,SKU00976,33703.81,0.0006
8,SKU04021,33584.48,0.0006
9,SKU09711,33301.50,0.0006


In [24]:
q3 = """
SELECT
  s.category,
  SUM(o.quantity * s.price) AS revenue,
  SUM(o.quantity * (s.price - s.cost)) AS gross_profit,
  ROUND(
    1.0 * SUM(o.quantity * (s.price - s.cost)) /
    NULLIF(SUM(o.quantity * s.price), 0),
    4
  ) AS gross_margin_pct
FROM orders_clean o
JOIN sku_clean s ON s.sku_id = o.sku_id
GROUP BY s.category
ORDER BY gross_profit DESC;
"""
pd.read_sql(q3, conn)


,category,revenue,gross_profit,gross_margin_pct
0,Automotive,5895787.76,2702165.21,0.4583
1,Pet Supplies,5698388.08,2655986.58,0.4661
2,Health & Beauty,5686930.50,2618072.48,0.4604
3,Home & Kitchen,5603308.98,2573934.82,0.4594
4,Apparel,5481246.73,2533125.85,0.4621
5,Grocery,5489373.35,2502164.91,0.4558
6,Office Supplies,5436157.88,2494690.30,0.4589
7,Toys,5343491.54,2442127.23,0.4570
8,Sports,5092701.22,2368982.36,0.4652
9,Electronics,5111741.07,2362335.17,0.4621


## 2) Fulfillment & Shipping Performance
Goal: Identify how shipping choices and regions affect delivery and fulfillment timing.


In [25]:
q4 = """
SELECT
  region,
  shipping_method,
  COUNT(*) AS orders,
  ROUND(AVG(days_to_fulfill), 2) AS avg_days_to_fulfill,
  ROUND(AVG(delivery_time_days), 2) AS avg_delivery_days
FROM orders_clean
GROUP BY region, shipping_method
ORDER BY region, avg_delivery_days;
"""
pd.read_sql(q4, conn)


,region,shipping_method,orders,avg_days_to_fulfill,avg_delivery_days
0,Central,Next-Day,1649,0.34,1.67
1,Central,Expedited,3256,0.74,2.50
2,Central,Standard,11123,1.50,3.51
3,East,Next-Day,1617,0.34,1.18
4,East,Expedited,3207,0.75,1.69
5,East,Standard,11318,1.49,2.52
6,Midwest,Next-Day,1640,0.33,1.70
7,Midwest,Expedited,3130,0.76,2.55
8,Midwest,Standard,11244,1.49,3.51
9,South,Next-Day,1569,0.33,1.19


In [26]:
q5 = """
SELECT
  s.category,
  o.shipping_method,
  COUNT(*) AS orders,
  ROUND(AVG(o.delivery_time_days), 2) AS avg_delivery_days
FROM orders_clean o
JOIN sku_clean s ON s.sku_id = o.sku_id
GROUP BY s.category, o.shipping_method
HAVING COUNT(*) >= 20
ORDER BY avg_delivery_days DESC
LIMIT 10;
"""
pd.read_sql(q5, conn)


,category,shipping_method,orders,avg_delivery_days
0,Electronics,Standard,5223,3.35
1,Pet Supplies,Standard,5758,3.34
2,Grocery,Standard,5631,3.32
3,Health & Beauty,Standard,5733,3.32
4,Home & Kitchen,Standard,5807,3.32
5,Office Supplies,Standard,5680,3.31
6,Sports,Standard,5158,3.31
7,Automotive,Standard,5749,3.29
8,Toys,Standard,5484,3.29
9,Apparel,Standard,5806,3.28


## 3) Supplier Performance (Ranking + Risk List)
Goal: rank suppliers on reliability and isolate late-risk shipments.


In [27]:
q6 = """
WITH supplier_kpi AS (
  SELECT
    supplier_id,
    COUNT(*) AS shipments,
    ROUND(AVG(fill_rate), 3) AS avg_fill_rate,
    ROUND(AVG(CASE WHEN lead_variance <= 0 THEN 1.0 ELSE 0.0 END), 3) AS on_time_rate,
    ROUND(AVG(lead_variance), 2) AS avg_lead_variance
  FROM supplier_clean
  GROUP BY supplier_id
)
SELECT
  *,
  DENSE_RANK() OVER (ORDER BY on_time_rate DESC) AS on_time_rank,
  DENSE_RANK() OVER (ORDER BY avg_fill_rate DESC) AS fill_rank
FROM supplier_kpi
ORDER BY on_time_rank, fill_rank
LIMIT 20;
"""
pd.read_sql(q6, conn)


,supplier_id,shipments,avg_fill_rate,on_time_rate,avg_lead_variance,on_time_rank,fill_rank
0,SUP039,527,0.983,0.679,-0.71,1,4
1,SUP040,469,0.982,0.670,-0.60,2,5
2,SUP060,485,0.985,0.666,-0.62,3,2
3,SUP015,480,0.982,0.662,-0.63,4,5
4,SUP004,495,0.981,0.661,-0.47,5,6
5,SUP046,514,0.980,0.661,-0.44,5,7
6,SUP059,539,0.983,0.659,-0.58,6,4
7,SUP056,507,0.982,0.659,-0.58,6,5
8,SUP003,491,0.978,0.654,-0.63,7,9
9,SUP029,499,0.983,0.653,-0.45,8,4


In [28]:
q7 = """
SELECT
  supplier_id,
  shipment_id,
  order_date,
  promised_lead_time,
  actual_lead_time,
  lead_variance,
  delivery_status,
  units_ordered,
  units_received,
  fill_rate
FROM supplier_clean
ORDER BY lead_variance DESC
LIMIT 30;
"""
pd.read_sql(q7, conn)


,supplier_id,shipment_id,order_date,promised_lead_time,actual_lead_time,lead_variance,delivery_status,units_ordered,units_received,fill_rate
0,SUP025,SHP027564,2022,11,23,12,Late,485,485,1.000000
1,SUP015,SHP017133,2022,5,16,11,Late,175,141,0.805714
2,SUP060,SHP028206,2023,19,30,11,Late,66,48,0.727273
3,SUP014,SHP000839,2022,16,26,10,Late,101,101,1.000000
4,SUP019,SHP001173,2022,19,29,10,Late,317,317,1.000000
5,SUP021,SHP003330,2023,20,30,10,Late,398,398,1.000000
6,SUP059,SHP007421,2024,12,22,10,Late,278,278,1.000000
7,SUP007,SHP010178,2024,19,29,10,Late,299,299,1.000000
8,SUP037,SHP012191,2024,13,23,10,Late,391,391,1.000000
9,SUP004,SHP012215,2023,8,18,10,Late,139,139,1.000000


## 4) Warehouse Operations (Efficiency vs Quality)
Goal: rank warehouses by productivity and quality (errors/rework) using a scorecard approach.


In [29]:
q8 = """
WITH wh AS (
  SELECT
    warehouse_id,
    COUNT(*) AS days,
    ROUND(AVG(units_per_labor_hour), 3) AS avg_units_per_labor_hour,
    ROUND(AVG(error_rate), 4) AS avg_error_rate,
    ROUND(AVG(rework), 2) AS avg_rework,
    ROUND(AVG(receiving_volume), 2) AS avg_receiving_volume,
    ROUND(AVG(outbound_volume), 2) AS avg_outbound_volume
  FROM warehouse_clean
  GROUP BY warehouse_id
)
SELECT
  *,
  DENSE_RANK() OVER (ORDER BY avg_units_per_labor_hour DESC) AS efficiency_rank,
  DENSE_RANK() OVER (ORDER BY avg_error_rate ASC) AS quality_rank
FROM wh
ORDER BY efficiency_rank, quality_rank;
"""
pd.read_sql(q8, conn)


,warehouse_id,days,avg_units_per_labor_hour,avg_error_rate,avg_rework,avg_receiving_volume,avg_outbound_volume,efficiency_rank,quality_rank
0,WFC,1096,1.917,0.0034,1.18,881.05,984.64,1,2
1,CFC,1096,1.916,0.0033,1.16,878.81,987.59,2,1
2,EFC,1096,1.909,0.0034,1.21,875.34,991.65,3,2
3,SFC,1096,1.894,0.0034,1.23,876.26,982.65,4,2
4,MFC,1096,1.885,0.0034,1.16,880.85,987.89,5,2


In [30]:
q9 = """
SELECT
  warehouse_id,
  ROUND(AVG(outbound_volume - receiving_volume), 2) AS avg_outbound_minus_receiving,
  ROUND(AVG(outbound_volume), 2) AS avg_outbound,
  ROUND(AVG(receiving_volume), 2) AS avg_receiving
FROM warehouse_clean
GROUP BY warehouse_id
ORDER BY avg_outbound_minus_receiving DESC;
"""
pd.read_sql(q9, conn)


,warehouse_id,avg_outbound_minus_receiving,avg_outbound,avg_receiving
0,EFC,116.31,991.65,875.34
1,CFC,108.78,987.59,878.81
2,MFC,107.04,987.89,880.85
3,SFC,106.39,982.65,876.26
4,WFC,103.59,984.64,881.05
